<a href="https://colab.research.google.com/github/jaunty0216/ForSophomoreCourse/blob/main/%E7%AC%AC%E4%B8%89%E9%80%B1_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 目標
這次的目標：**連結到GoogleSheet，一樣練習存取或讀寫，但試著把資料拿給gemini的AI做處理。**

---



# 初始化
**要先取得apikey和模型**

In [1]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

import google.generativeai as genai
genai.configure(api_key = GOOGLE_API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")

# 取得資料並處理
**會用到這些函式庫喔**

1.  from google.colab import auth
2.  from google.auth import default
3.  from gspread_dataframe import set_with_dataframe
4.  import gspread
5.  import pandas as pd




**運作的邏輯**

1.   登入，驗證帳號
2.   載入google sheet與相關的函式庫
3.   讀取google sheet的資料
4.   把資料給gemini
5.   把處理好的資料重新寫回google sheet



In [3]:
# 登入，驗證帳號
from google.colab import auth

try:
  auth.authenticate_user()
  print("successfully accessed!")
except:
  print("error")

successfully accessed!


In [17]:
# 載入google sheet與相關的函式庫
from google.auth import default
import gspread

creds ,_ = default()
gc = gspread.authorize(creds)
print(f"this is the creds:",creds)
# 這個print只是展示用

try:
  ProgrammingWeek3 = gc.open_by_url("https://docs.google.com/spreadsheets/d/1BMZSQrSbS-4iYG5zsUjNAOhusNXGXIKYEezwK6Ad9GQ/edit?usp=drivesdk")
  NameSheet = ProgrammingWeek3.get_worksheet(0)
  print("successfully accessed the worksheet!")
except:
  print("error")

this is the creds: <google.auth.compute_engine.credentials.Credentials object at 0x785240a4de10>
successfully accessed the worksheet!


In [43]:
# 讀取google sheet的資料
import pandas as pd

NameSheetData = NameSheet.get_all_values()
df_NameSheetData = pd.DataFrame(NameSheetData)
print(df_NameSheetData)

        0
0    John
1  Sherry
2  Tomido
3  Jaunty
4    Hare


In [44]:
# 把資料給gemini
prompt_1 = f"you're an HR of adventurer association, you have to give each person on the list a career according to their names. Here is the list",df_NameSheetData.to_string()
result = model.generate_content(prompt_1)
print(result.text)

Okay, let's assign roles based on the adventurer's names, keeping in mind the need for a balanced team:

* **John:**  **Field Commander/Leader**.  A classic, strong name suggesting reliability and leadership qualities.

* **Sherry:** **Scout/Ranger**.  The name has a light, quick feel, fitting for someone skilled in reconnaissance and stealth.

* **Tomido:** **Heavy Weapons Specialist/Brawler**. The name sounds strong and imposing, implying a character proficient in close-quarters combat or handling powerful weaponry.

* **Jaunty:** **Rogue/Thief**.  "Jaunty" suggests a mischievous and nimble individual, perfect for infiltration and cunning tactics.

* **Hare:** **Tracker/Scout (support)**.  Evokes speed and keen senses, ideal for tracking and providing support to the team's scouts.


This provides a diverse team with complementary skills.  Of course, a proper evaluation of their actual skills would be needed for accurate placement!



In [54]:
# 把處理好的資料重新寫回google sheet
from io import StringIO
from gspread_dataframe import set_with_dataframe

prompt_2 = f'Summarize the contents below,Just write the names and their own careers only. ',result.text
FilteredData = model.generate_content(prompt_2)

df_FilteredData = pd.read_csv(StringIO(FilteredData.text))
print(df_FilteredData)
AssignedSheet = ProgrammingWeek3.get_worksheet(1)
set_with_dataframe(AssignedSheet,df_FilteredData)

               John: Field Commander/Leader
0                      Sherry: Scout/Ranger
1  Tomido: Heavy Weapons Specialist/Brawler
2                       Jaunty: Rogue/Thief
3                       Hare: Tracker/Scout


# 結尾
我學到了啥

其實也沒有特別學到新的東西

主要是本來就學會的知識串接在了一起


---


像是本來就會讀與寫google sheet的資料

也會控制gemini

這一次就只是把從google sheet讀來的資料

餵給gemini之後

把結果整理好之後

寫到google sheet而已

但其實還是有學到新東西的

像是：


1.   from io import StringIO
  *   StringIO的概念：不用額外在電腦上建立一個臨時文件，屬於是憑空捏造出一個物件，然後這個物件可以模擬成不同檔案來做處裡。比方說今天要讀取csv檔案，那我可以利用StringIO創造出一個用來模擬csv檔案的物件，嚴格來說這物件不是csv檔，但傳給能讀取csv檔案的函數，也能使其正常運作。
  *   StringIO的用法：StringIO(要被模擬的資料)
2.   pd.read_csv
  *   pandas裡用來檢視csv檔案的工具

